### **Great Expectations**

Pada notebook ini, dilakukan data validasi dengan menggunakan python libary bernama great_expectations. Adapun ekspektasi yang dilakukan terhadap data adalah sebagai berikut:
+ to be unique
+ to be between min_value and max_value
+ to be in set
+ to be in type list
+ to be Dateutil parseable
+ Most Common value to be in set
+ Proportion of unique value to be between.

Kolom yang diterapkan pemeriksaan dipilih berdasarkan kebutuhan tujuan analisa. Sehingga tidak semua kolom dilakukan validasi great expectation. Namun terdapat beberapa kolom yang dilakukan validasi lebih dari 1 kali dengan expecation yang berbeda.

Data Context

In [19]:
from great_expectations.data_context import FileDataContext

In [20]:
context = FileDataContext.create(project_root_dir='./')

Connect to Datasource

In [21]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'crimes_data'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'crimes_in_la'
path_to_data = 'dags/P2M3_ardianto_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

Expectation Suits

In [22]:
# Creat an expectation suite
expectation_suite_name = 'expectation-crimes_dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,dr_no,date_occ,time_occ,area,area_name,rpt_dist_no,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,weapon_used_cd,weapon_desc,status,status_desc,location,lat,lon
0,200406624,2020-03-08,4,4,Hollenbeck,471,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",0344 0345,0,X,X,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,500 S ANDERSON ST,34.0412,-118.2250
1,200506784,2020-03-05,4,5,Harbor,521,330,BURGLARY FROM VEHICLE,0344 0377 1307 0321 1402 1609,39,F,H,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,400 WESTMONT DR,33.7638,-118.2939
2,200307774,2020-03-02,16,3,Southwest,357,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",0344 2032,38,M,B,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,VERMONT AV,34.0231,-118.2915
3,200412941,2020-09-19,6,4,Hollenbeck,497,310,BURGLARY,0325 0913 1803,0,X,X,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,3400 E 15TH ST,34.0158,-118.2059
4,200506610,2020-03-02,14,5,Harbor,557,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",1300 0329 0216,63,F,W,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,3RD,33.7415,-118.2901


##### **1. To be Unique**

Pada bagian ini, dilakukan pemeriksaan terhadap kolom dr_no atau (Division of Records Number) sebab kolom tersebut merupakan unique identfier setiap data baris dan tidak boleh terdplikasi meskipun dengan isi kolom lain dari baris itu berbeda. 

In [23]:
validator.expect_column_values_to_be_unique('dr_no')


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 495410,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

> Kolom dr_no lulus uji karena tidak ada yang duplikat

##### **2. To be between min_value and max_value**

Untuk bagian ini, dilakukan pemeriksaan terhadap kolom time_occ (jam), dan kolom crm_cd (crimes_codes). 

In [24]:
# jam tentunya tidak boleh kurang dari 0 dan 23
validator.expect_column_values_to_be_between(column='time_occ', min_value=0, max_value=23)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 495410,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

In [25]:
# untuk crm_codes value harus berada di antara 110 (homicide) dan 956 (Conspiracy to kill, kidnap, maim, or injure persons or damage property in a foreign country)

validator.expect_column_values_to_be_between(column='crm_cd', min_value=110, max_value=956)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 495410,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

> Kolom time_occ dan crm_cd lulus uji

##### **3. To be in Set**

Untuk to be in Set, diterapkan pada kolom vict_sex (gender) dan status (status dari case). Dengan rincian sebagai berikut:
1. vict_sex : F(female), M(Male), X(unknown), H(Diasumsikan sebagai teridentifikasi Homosexual)
2. staus : IC (Investigation Continues), AO (Adult Other), AA (Adult Arrest), JA (Juvenile Arrest), JO (Juvenile Other). 

In [26]:
validator.expect_column_values_to_be_in_set('vict_sex', ["F", "M", "H", "X"])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 495410,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

In [27]:
validator.expect_column_values_to_be_in_set(['status'], ["AO", "AA", "IC", "JA", "JO"])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 495410,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [
      "status"
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

> Kolom vict_sex dan status lulus uji

##### **4. To be type list**

Untuk bagian ini, pemeriksaan dilakukan terhadap kolom longitude dan latitude dengan alasan bahwa kolom tersebut harus bertipe float atau integer untuk dapat digunakan sebagai geopoint pada kibana. 

In [28]:
validator.expect_column_values_to_be_in_type_list('lon', ['float', 'int64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "result": {
    "observed_value": "float64"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

In [29]:
validator.expect_column_values_to_be_in_type_list('lat', ['float', 'int64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "result": {
    "observed_value": "float64"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

> Kedua kolom lat dan lon lulus uji

##### **5. To be Dateutil parseable**

Expectation ini digunakan untuk memeriksa validitas dari suatu kolom date/tanggal apakah kolom tersebut bisa diparse menjadi tanggal atau tidak.

In [30]:
validator.expect_column_values_to_be_dateutil_parseable("date_occ")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "result": {
    "element_count": 495410,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

> Kolom date_occ lulus uji

##### **6. Most Common Value to be in Set**

Salah satu goal analysis adalah mencari tau lebih mengenai korban dari kejahatan salah satunya adalah mengetahui jumlah korban berdasarkan gender laki-laki atau perempuan. Untuk itu, diharapkan kolom vict_sex memliki most common value F (female) atau M (Male) bukan malah X atau H.

In [31]:
validator.expect_column_most_common_value_to_be_in_set('vict_sex', ['M', 'F'])

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "result": {
    "observed_value": [
      "M"
    ]
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

##### **7. Proportion of Unique values to be between**

Pada ekspektasi ke tujuh ini, dilakukan pemeriksaan terhadap crm_cd dimana data merupakan categorical numerik untuk setiap kode crimes/kejahatan. Expextation ini memeriksa proporsi dari unique values untuk beradad di antara minimum value dan maximum value.
Contoh, pada sebuah kolom berisi [1, 2, 2, 3, 3, 3, 4, 4, 4, 4], terdapat 4 unique values dan 10 total values berarti proporsi unique valuenya 0.4.

Untuk itu, dikarenakan crm_cd adalah data dengan tipe categorical numeric, diharapkan unique value yang tersedia tidak lebih dari 0.001 mengingat kode crimes hanya tersedia dari rentang 110 - 956 atau sekitar 900an unique value sedangkan jumlah baris data terdapat 490K lebih. 

In [32]:
validator.expect_column_proportion_of_unique_values_to_be_between(column='crm_cd', min_value=0, max_value=0.001)

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "result": {
    "observed_value": 0.00027452009446720895
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

> Column crm_cd lulus validasi